# Network Analysis

In [ ]:
import sys
import numpy as np
import tabulate
import time
import copy

import logging

from sklearn.datasets import make_sparse_spd_matrix
import matplotlib.pyplot as plt
import os
import sys
import re
import glob
import ast
import os.path as op
import pickle
import scipy
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import scipy as sp

from collections import defaultdict

from copy import deepcopy
import copy

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, FastICA
from sklearn.utils import resample
from sklearn.covariance import GraphicalLassoCV
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LinearRegression

from itertools import combinations
from scipy.stats import ttest_ind

import scipy.stats as stats

import networkx as nx
import glob, os
from functools import partial

from scipy.stats import pearsonr

import pickle

import warnings
warnings.filterwarnings("ignore")

## Load data

In [ ]:
dataset = 'train'
test = False if dataset == 'train' else False

Load training data

In [ ]:
results_ern_df = pd.read_pickle(f"../data/models_pickles_new_dass/ern_models_{dataset}.pkl")
results_ern_lat_demo_df = pd.read_pickle(f"../data/models_pickles_new_dass/ern_cov_fal_models_{dataset}.pkl")
results_crn_df = pd.read_pickle(f"../data/models_pickles_new_dass/crn_models_{dataset}.pkl")
results_crn_lat_demo_df = pd.read_pickle(f"../data/models_pickles_new_dass/crn_cov_fal2_models_{dataset}.pkl")

Load testing data

In [ ]:
results_ern_test_df = pd.read_pickle(f"../data/models_pickles_new_dass/ern_models_test.pkl")
results_ern_lat_demo_test_df = pd.read_pickle(f"../data/models_pickles_new_dass/ern_cov_fal_models_test.pkl")
results_crn_test_df = pd.read_pickle(f"../data/models_pickles_new_dass/crn_models_test.pkl")
results_crn_lat_demo_test_df = pd.read_pickle(f"../data/models_pickles_new_dass/crn_cov_fal2_models_test.pkl")

# fill nan
results_crn_lat_demo_test_df['e_LT_F2_C'] = results_crn_lat_demo_test_df['e_LT_F2_C'].fillna(results_crn_lat_demo_test_df['e_LT_F2_C'].mean())

Define names mapping and order

In [ ]:
mapping = {
    "RRQ": "1Q",
    "DASS-21 Stress": "2Q",
    "DASS-21 Dep": "3Q",
    "STAI-T": "4Q",
    "STAI-S Diff": '5Q',
    "BIS": "6Q",
    "OBSESS": "7Q",
    "HOARD": "8Q",
    "ORD": "9Q",
    "CHECK": "10Q",
    "WBSI": "11Q",
    "IUS-P": "12Q",
    "IUS-I": "13Q",
    "SES": "14Q",
    'BAS_D': "15Q",
    'BAS_PRZY': "16Q",
    'BAS_NAG': "17Q",
    'INDEC_F': "18Q",
    'PUN': "19Q",
    'REW': "20Q",
    'HARM': "21Q",
    'T-CTR': "22Q",
    "OT": "23Q",
    'OB_PERF': "24Q",
    'PS': "25Q",
    'G_SE': "26Q",
    'AMB': "27Q",
    'PRED': "28Q",
    'STAND': "29Q",   
    "Age": "1C",
    "Handness": "2C",
    'e_ERN': "1E",
    'e_LT_F': "4C",
    'performance': "3C",
    'e_CRN': "2E",
    'e_LT_F2_C': "4C",
    'performance': "3C",
}

mapping_order = {
    '1E': 0,
    '2E': 1,
    '1Q':2,
    '2Q':3,
    '3Q':4,
    '4Q':5,
    '5Q':6,
    '6Q':7,
    '7Q':8,
    '8Q':9,
    '9Q':10,
    '10Q':11,
    '11Q':12,
    '12Q':13,
    '13Q':14,
    '14Q':15,
    '15Q':16,
    '16Q':17,
    '17Q':18,
    '18Q':19,
    '19Q':20,
    '20Q':21,
    '21Q':22,
    '22Q':23,
    '23Q':24,
    '24Q':25,
    '25Q':26,
    '26Q':27,
    '27Q':28,
    '28Q':29,
    '29Q':30,
    '1C':31,
    '2C':32,
    '3C':33,
    '4C':34,   
}

Prepare datasets

In [ ]:
columns_to_drop_covs = ['WASH', 'NEU', 'Sex']
columns_to_drop = ['WASH', 'NEU']

# train dataset
ern_data_df_ = results_ern_df.drop(columns=columns_to_drop).rename(columns=mapping)
ern_cov_fal_data_df_ = results_ern_lat_demo_df.drop(columns=columns_to_drop_covs).rename(columns=mapping)
crn_data_df_ = results_crn_df.drop(columns=columns_to_drop).rename(columns=mapping)
crn_cov_fal2_data_df_ = results_crn_lat_demo_df.drop(columns=columns_to_drop_covs).rename(columns=mapping)

ern_data_df_diff_ = ern_data_df_
ern_cov_fal_data_df_diff_ = ern_cov_fal_data_df_

# test dataset
ern_data_df_test_ = results_ern_test_df.drop(columns=columns_to_drop).rename(columns=mapping)
ern_cov_fal_data_df_test_ = results_ern_lat_demo_test_df.drop(columns=columns_to_drop_covs).rename(columns=mapping)
crn_data_df_test_ = results_crn_test_df.drop(columns=columns_to_drop).rename(columns=mapping)
crn_cov_fal2_data_df_test_ = results_crn_lat_demo_test_df.drop(columns=columns_to_drop_covs).rename(columns=mapping)

ern_data_df_diff_test_ = ern_data_df_test_
ern_cov_fal_data_df_diff_test_ = ern_cov_fal_data_df_test_


datasets_train = [ern_data_df_, ern_cov_fal_data_df_, crn_data_df_, crn_cov_fal2_data_df_, ern_data_df_diff_, ern_cov_fal_data_df_diff_]
datasets_test = [ern_data_df_test_, ern_cov_fal_data_df_test_, crn_data_df_test_, crn_cov_fal2_data_df_test_, ern_data_df_diff_test_, ern_cov_fal_data_df_diff_test_]

### Read estimated graphs

In [ ]:
dir_ = f"../data/graphs/{dataset}/"

graphs = []

for file in sorted(glob.glob(dir_ + "*.gml")):
    print(file)
    graph = nx.read_gml(file)
    graphs.append(graph)
    
graphs

## Network analysis

### Define custom network measures 

In [ ]:
def calculate_nodes_predictability(X, G):
    explained_variance = dict()
        
    for node in G.nodes():
        y_ = X[[node]]

        neighbors = list(G.neighbors(node))

        X_ = X.loc[:, neighbors]

        lm = LinearRegression()
        lm.fit(X_, y_)

        score = lm.score(X_,y_)
        explained_variance[node] = score

    return explained_variance

In [ ]:
def current_flow_closeness_centrality(G, weight):
    G_ = copy.deepcopy(G.copy())
    for u, v, data in G_.edges(data=True):
        data['weight'] = abs(data['weight'])
        
    measure = nx.current_flow_closeness_centrality(G_, weight=weight)
    return measure

### Perform network analysis

In [ ]:
network_analysis_results_df = pd.DataFrame()

for i in range(0,6):
    G = copy.deepcopy(graphs[i])
    dataset_train = datasets_train[i]
    dataset_test = datasets_test[i]

    measures = [
            (calculate_nodes_predictability, {'X': dataset_test}), 
            (nx.degree_centrality, {}),
            (nx.closeness_centrality, {}),
            (current_flow_closeness_centrality, {'weight': 'weight'}),
            (nx.betweenness_centrality, {'weight': 'weight'}),
            (nx.current_flow_betweenness_centrality, {'weight': 'weight'}),
            (nx.load_centrality, {})
    ]
    
    for measure, measure_parameters in measures:
        G = copy.deepcopy(graphs[i])
        network_measure = measure(G = G, **measure_parameters)
        network_measure = {k: v for k, v in sorted(network_measure.items(), key=lambda item: item[0], reverse=True)}
        network_measure_nodes = network_measure.keys()
        network_measure_values = network_measure.values()
        network_measure_df = pd.DataFrame({
            'node': network_measure_nodes,
            'value' : network_measure_values,
            'graph' : [i] * len(network_measure_nodes),
            'measure' : [measure.__name__] * len(network_measure_nodes),
        })
        network_analysis_results_df = pd.concat([network_analysis_results_df, network_measure_df], ignore_index = True)

In [ ]:
with pd.option_context('display.max_colwidth', None, 'display.max_columns', None,  'display.max_rows', None,):
    display(network_analysis_results_df.groupby(['graph', 'measure']).apply(lambda a: a.drop(['graph', 'measure'], axis=1)[:]))

Save the results

In [ ]:
network_analysis_results_df.to_pickle('../data/network_analysis/base_estimates/network_analysis_results_with_diff_graphs.pkl')
network_analysis_results_df.to_csv('../data/network_analysis/base_estimates/network_analysis_results_with_diff_graphs.csv')

Save the results in pleasant form

In [ ]:
network_analysis_results_df = pd.read_pickle('../data/network_analysis/base_estimates/network_analysis_results_with_diff_graphs.pkl')
network_analysis_results_df.head()

In [ ]:
df = network_analysis_results_df.copy()
df = df[df['measure'].isin(['calculate_nodes_predictability', 'degree_centrality', 'current_flow_closeness_centrality'])]

df['node'] = df['node'].replace(mapping)
df = df.sort_values(by=['node'], key=lambda x: x.map(mapping_order))
df = df.pivot_table(index='node', columns=['graph', 'measure'])
df = df.sort_values(by=['node'], key=lambda x: x.map(mapping_order))

# reorder and rename network measures
net_colums_names_mapper = {
    'calculate_nodes_predictability' : 'P',
    'current_flow_closeness_centrality': 'C-F-C-C',
    'degree_centrality' : 'D-C',    
}

df = df.rename(columns=net_colums_names_mapper)

net_colums_names_order = ['P', 'D-C','C-F-C-C']
new_cols = df.columns.reindex(net_colums_names_order, level=2)

df = df.reindex(columns=new_cols[0])

with pd.option_context(
    'display.max_colwidth', None, 
    'display.max_columns', None,  
    'display.max_rows', None, 
    'display.float_format', '{:.3f}'.format):
    display(df)
    
df.to_csv('../data/network_analysis/base_estimates/network_analysis_results_with_diff_graphs_easy_view.csv')

### Visualize the results

In [ ]:
cm = 1/2.54
dpi = 500

sns.set_style("whitegrid")

plt.rcParams['figure.dpi'] = dpi
plt.rcParams['ytick.labelsize'] = 6
plt.rcParams['xtick.labelsize'] = 6
plt.rcParams['axes.labelsize'] = 6
plt.rcParams['axes.titlesize'] = 6
plt.rcParams["axes.edgecolor"] = ".15"
plt.rcParams["axes.linewidth"]  = 0.6
plt.rcParams['lines.linewidth'] = 0.6

for i in range(0,6):
    plt.figure(i)

    df = network_analysis_results_df.copy()
    df = df[df['measure'].isin(['calculate_nodes_predictability', 'degree_centrality', 'current_flow_closeness_centrality'])]
    df.node = df.node.astype(str)
    df_ = df[df['graph'] == i]

    df_['node'] = df_['node'].replace(mapping)
    df_ = df_.sort_values(by=['node'], key=lambda x: x.map(mapping_order))

    
    g = sns.FacetGrid(
        df_, 
        col='measure',
        col_order=['calculate_nodes_predictability', 'degree_centrality', 'current_flow_closeness_centrality'],
        sharex=False,
        height=8*cm,
        aspect=0.3,
        despine=False,
        gridspec_kws={"wspace":0.2}
    )

    g.map(
        sns.pointplot, 
        'value',
        'node',
        errorbar=None,
        color='black',
    )

    axes = g.axes.flatten()
    axes[0].set_title("Predictability")
    axes[1].set_title("D-Centrality")
    axes[2].set_title("C-F Closeness")
    
    g.set_axis_labels(x_var="")

    fig = g.fig
    # fig.savefig(f'data/network_analysis/base_estimates/network_analysis_dpi_{dpi}_graph_{i}_abs.png', bbox_inches='tight')
    
plt.show()